# (Bonus) Asynchronous LLM calls

For evaluation, we need to run multiple articles through the LLM to get a good sense of its performance.
Currently, our entire pipeline is synchronous, which means we have to wait for each article to be processed before we can move on to the next one. Furthermore, even within our pipeline, we have to wait for each task to be completed before we can move on to the next one.

Asynchronous calls can be immensely helpful here, as they allow us to send multiple requests to the LLM at once. For evaluation, this means that we can pass multiple articles through the LLM system at the same time. And for our pipeline, this means that we can even gather information about different businesses in parallel.

We won't build out the full pipeline like this in this training, but as a bunus exercise, you can try to implement async calls to get the business information for multiple companies at once.


In [ ]:
import asyncio
from typing import List

from pydantic import BaseModel

from llmops_training.news_reader.generation import generate_object, generate_object_async
from llmops_training.news_reader.extraction import (
    BusinessSpecificInfo,
    format_prompt, 
    extract_business_info,
    extract_businesses_involved,
)
from llmops_training.news_reader.data import get_bbc_news_sample

In [ ]:
task1 = "Extract structured info from: 'The apple was blue and the sky was red'"
task2 = "Extract structured info from: 'The apple was green and the sky was blue'"
task3 = "Extract structured info from: 'The apple was red and the sky was green'"
task4 = "Extract structured info from: 'The apple was yellow and the sky was purple'"
task5 = "Extract structured info from: 'The apple was actually an orange and the sky was apple colored'"

tasks = [task1, task2, task3, task4, task5]

class MyObject(BaseModel):
    apple_color: str
    sky_color: str

Check out the difference in speed if we run the following synchoronous or asynchronous!

In [ ]:
# Synchronous

results = []
for task in tasks:
    results.append(generate_object(task, MyObject))   
results

In [ ]:
# Asynchronous

running_tasks = []
for task in tasks:
    running_tasks.append(generate_object_async(task, MyObject)) 

results = await asyncio.gather(*running_tasks)
results

That was 5x as fast! 🚀

We can extract business info from articles in an asynchronous way as well:

In [ ]:
async def extract_business_specific_info_async(
    prompt_template: str, article: str, business: str, **kwargs
) -> BusinessSpecificInfo:
    prompt = format_prompt(prompt_template, article=article, business=business)
    return await generate_object_async(prompt, BusinessSpecificInfo, **kwargs)


def is_business_we_care_about(business: str) -> bool:
    return True  # Some logic here

async def extract_business_info_async(
    businesses_involved_prompt_template: str, 
    business_specific_prompt_template: str, 
    article: str, 
    **kwargs,
) -> List[BusinessSpecificInfo]:
    businesses = extract_businesses_involved(
        businesses_involved_prompt_template, article
    ).businesses
    business_info_tasks = []
    for business in businesses:
        if is_business_we_care_about(business):
            business_info_tasks.append(
                extract_business_specific_info_async(
                    business_specific_prompt_template, article, business, **kwargs
                )
            )
    business_info = await asyncio.gather(*business_info_tasks)

    return business_info

In [ ]:
businesses_involved_prompt_template = "Extract structured info from the following article:\n{article}"
business_specific_prompt_template = "Extract structured information for the business '{business}' from the following article:\n{article}"

Now let's check the difference with what we had before:

In [ ]:
data = get_bbc_news_sample()
article = data[data["is_business"]].iloc[0]["article"]

In [ ]:
extract_business_info(
    businesses_involved_prompt_template, 
    business_specific_prompt_template, 
    article
)

In [ ]:
await extract_business_info_async(
    businesses_involved_prompt_template, 
    business_specific_prompt_template, 
    article
)

> **Exercise** 📝
>
> - Integrate this into our app and check how the traces change! 🚀
> 